In [1]:
from allennlp.data.dataset_readers import DatasetReader, Seq2SeqDatasetReader
from allennlp.data.tokenizers import Tokenizer, WordTokenizer, CharacterTokenizer
from allennlp.data.fields import LabelField, TextField, Field
from allennlp.data import Instance
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer, TokenCharactersIndexer
from allennlp.data.iterators import BucketIterator

from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder

# preprocessing 
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

# encoder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.seq2seq_encoders import StackedSelfAttentionEncoder

# model
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator

# attention
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention

# other packages
from overrides import overrides
import pandas as pd
from collections import namedtuple
from sklearn.model_selection import train_test_split
import itertools
from allennlp.predictors import SimpleSeq2SeqPredictor

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim




In [2]:
torch.cuda.is_available()

True

In [3]:
Config = namedtuple('Config', [
                    'lazy',
                    'max_vocab_size', 
                    'batch_size', 
                    'lr', 
                    'epochs', 
                    'max_seq_len', 
                    'IN_EMBEDDING_DIM', 
                    'HIDDEN_DIM', 
                    'OUT_EMBEDDING_DIM', 
                    'CUDA_DEVICE'
])

In [5]:
config = Config(True,
               10000,
               64,
               10e-4,
               2,
               100,
               256,
               256,
               256,
               0 if torch.cuda.is_available() else -1)

In [7]:
reader = Seq2SeqDatasetReader(
    source_tokenizer = CharacterTokenizer(),
    target_tokenizer = CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')},
    lazy=config.lazy
)

In [28]:
raw_dataset = pd.read_csv('data/ru_train.csv')

In [29]:
raw_dataset['before'] = raw_dataset['before'].astype(str)
raw_dataset['after'] = raw_dataset['after'].astype(str)

In [30]:
df = raw_dataset[:].groupby('sentence_id').agg({
    'before': lambda x: ' '.join(x),
    'after': lambda x: ' '.join(x)
})

In [31]:
df_train, df_test = train_test_split(df)

In [32]:
df_train[:100000].to_csv('train_dataset.tsv', index=False, header=False, sep='\t')
df_test[:1000].to_csv('test_dataset.tsv', index=False, header=False, sep='\t')

In [33]:
train_dataset = reader.read('train_dataset.tsv',)
validation_dataset = reader.read('test_dataset.tsv')

# prepare vocabulary

In [34]:
vocab = Vocabulary.from_instances(train_dataset,
                                  min_count={'tokens': 3, 'target_tokens': 3}
                                 )

100000it [00:20, 4861.54it/s]


In [35]:
vocab_test = Vocabulary.from_instances(validation_dataset,
                                       min_count={'tokens': 3, 'target_tokens': 3}
                                      )

1000it [00:00, 5146.06it/s]


In [36]:
in_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=config.IN_EMBEDDING_DIM
                        )

In [37]:
source_embedder = BasicTextFieldEmbedder({"tokens": in_embedding})

In [38]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("source_tokens", "num_tokens")],
                         )

In [39]:
iterator.index_with(vocab)

# prepare model

In [40]:
## get simple encoder
encoder = PytorchSeq2SeqWrapper(torch.nn.LSTM(config.IN_EMBEDDING_DIM, 
                                              config.HIDDEN_DIM, 
                                              batch_first=True))

In [41]:
max_decoding_steps = 100

model = SimpleSeq2Seq(vocab, 
                      source_embedder, 
                      encoder, 
                      max_decoding_steps,
                      target_embedding_dim=config.OUT_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      beam_size=8,
                      use_bleu=True)

In [42]:
model.cuda(config.CUDA_DEVICE)

SimpleSeq2Seq(
  (_source_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding()
  )
  (_encoder): PytorchSeq2SeqWrapper(
    (_module): LSTM(256, 256, batch_first=True)
  )
  (_target_embedder): Embedding()
  (_decoder_cell): LSTMCell(256, 256)
  (_output_projection_layer): Linear(in_features=256, out_features=380, bias=True)
)

In [43]:
optimizer = optim.Adam(model.parameters())

In [44]:
# training

In [45]:
torch.cuda.is_available()

True

In [ ]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=1,
                  cuda_device=config.CUDA_DEVICE)

for i in range(config.epochs):
    print(f'Epoch: {i+1}')
    trainer.train()

    predictor = SimpleSeq2SeqPredictor(model, reader)

    for instance in itertools.islice(validation_dataset, 10):
        print('SOURCE:', ''.join(instance.fields['source_tokens'].tokens))
        print('GOLD:', ''.join(instance.fields['target_tokens'].tokens))
        print('PRED:', ''.join(predictor.predict_instance(instance)['predicted_tokens']))

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1


loss: 2.6937 ||: : 378it [02:15,  2.29it/s]                   